In [ ]:
!python ../setup.py build 
!python ../setup.py build_ext --inplace

In [1]:
import os, sys
sys.path.append("..")

# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [2]:
import opttrot

In [3]:
from opttrot.pauli_c import get_paulilist_from_coefs

In [4]:
from opttrot.pauli_c import PauliElement

In [5]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [7]:
from importlib import reload
import paulipoly
reload(paulipoly)
from paulipoly import PauliPoly as NewPauliPoly
#import ten_con
#reload(ten_con)
#from ten_con import _mat_to_coef_mat, _coef_to_mat

In [8]:
from scipy.sparse import coo_matrix, csr_matrix

In [9]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [20]:
from copy import deepcopy

In [21]:
qubits = 2
def get_test_hermit(qubits, tol= 0.5):
    ndim = int(2**qubits)
    A = np.random.rand(ndim, ndim)
    A[A<tol] = 0.
    H = A + 1j*A
    H = H.conj().T@H
    return H


In [22]:
A = np.sqrt(get_test_hermit(qubits, 0.0))
B = np.sqrt(get_test_hermit(qubits, 0.8))


In [12]:
test_ppoly = PauliPoly.from_matrix(A)

In [13]:
#plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix
cmat = csr_matrix(coef_mat)
100*cmat.data.size/(1024**2)

0.00152587890625

In [14]:
from opttrot.pauli_utils import mat_decompose, mat_compose

In [15]:
A

array([[0.98985323+0.j, 0.93684708+0.j, 1.12690697+0.j, 0.81213888+0.j],
       [0.93684708+0.j, 1.28596067+0.j, 1.41073572+0.j, 0.904974  +0.j],
       [1.12690697+0.j, 1.41073572+0.j, 1.93375635+0.j, 1.3168861 +0.j],
       [0.81213888+0.j, 0.904974  +0.j, 1.3168861 +0.j, 1.00680644+0.j]])

ModuleNotFoundError: No module named 'ten_con'

In [17]:
def identity(a):
    return  mat_compose(mat_decompose(a))

In [18]:
rea = identity(A)

In [19]:
rea

array([[0.98985323+0.j, 0.93684708+0.j, 1.12690697+0.j, 0.81213888+0.j],
       [0.93684708+0.j, 1.28596067+0.j, 1.41073572+0.j, 0.904974  +0.j],
       [1.12690697+0.j, 1.41073572+0.j, 1.93375635+0.j, 1.3168861 +0.j],
       [0.81213888+0.j, 0.904974  +0.j, 1.3168861 +0.j, 1.00680644+0.j]])

In [47]:
rea[2]

array([-1.31354936+0.j, -1.21544162+0.j,  1.64207507+0.j,  1.12583791+0.j])

In [48]:
A[2]

array([1.31354936+0.j, 1.21544162+0.j, 1.64207507+0.j, 1.12583791+0.j])

## Coef matrix to PauliPoly

In [ ]:
%%timeit
PauliPoly(coef_mat) # Naive

794 µs ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Hermit matrix to PauliPoly

In [ ]:
%%timeit
PauliPoly.from_matrix(mat)

15.3 ms ± 240 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
SparsePauliOp.from_operator(mat)

15 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Test

## PauliPoly to Hermit

In [ ]:
# PauliPoly
pp001 = PauliPoly.from_matrix(A)
pp002 = PauliPoly.from_matrix(B)

# Qiskit
spa001 = SparsePauliOp.from_operator(A)
spa002 = SparsePauliOp.from_operator(B)


In [ ]:
%%timeit
pp001.matrix

15.8 ms ± 497 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
spa001.to_matrix()

15.2 ms ± 274 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
pp002.matrix

15.7 ms ± 237 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
spa002.to_matrix()

15.7 ms ± 813 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## PauliElement generation

In [ ]:
ps = pp001.poly

In [ ]:
ps

array([PauliElement(n=8, weight=11.361267+(0.000000)j, IIIIIIII),
       PauliElement(n=8, weight=11.339729+(0.000000)j, IIIIIIIX),
       PauliElement(n=8, weight=11.335831+(-0.000000)j, IIIIIIXI), ...,
       PauliElement(n=8, weight=-0.000000+(0.024616)j, ZZZZZZYZ),
       PauliElement(n=8, weight=-0.000000+(0.034138)j, ZZZZZZZY),
       PauliElement(n=8, weight=0.024931+(0.000000)j, ZZZZZZZZ)],
      dtype=object)

In [ ]:
pp001.coef_matrix

matrix([[ 1.13612667e+01+6.16003139e-20j,
          1.13397290e+01+1.58148680e-18j,
          1.13358311e+01-3.52254625e-19j, ...,
          1.13354095e+01+3.51521445e-19j,
          1.13394664e+01-1.68055350e-18j,
          1.13377399e+01+3.13623140e-19j],
        [ 2.63857820e-19+3.41310799e-02j,
         -4.03569436e-02+1.73044697e-19j,
          7.17733694e-19+5.44267744e-02j, ...,
         -3.81759679e-02-2.38097439e-19j,
         -9.01175265e-20+4.88854616e-02j,
         -4.81928227e-02+1.79894977e-20j],
        [ 3.82952205e-19-1.38393144e-02j,
          1.82438546e-19-1.25717306e-02j,
          4.84406798e-02-8.93500470e-19j, ...,
         -4.24539480e-19+3.18263014e-04j,
          5.04355685e-02-9.21899191e-19j,
          6.60537155e-02-2.61969339e-19j],
        ...,
        [ 6.40999694e-19-1.04230381e-02j,
         -7.78282104e-03-1.02622624e-18j,
          4.37129288e-19-2.26382212e-02j, ...,
          1.55317423e-02+1.63452340e-20j,
         -3.06899589e-19+1.15281289e-02j

## Addition

**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [ ]:
%%timeit
pp001+pp002

1.2 ms ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
spa001+spa002

1.29 ms ± 25.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Mat multiplication

In [12]:
%%timeit
pp001@pp002

NameError: name 'pp001' is not defined

In [36]:
%%timeit
spa001@spa002 # Unable to multiplicate.

138 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Summary

|Opeartion|Faster|
|:-------:|:----:|
|o->mat   |      | 
|mat->o   |      | 
|Addition |      |
|Mat Mul  |      |
